In [0]:
# Read data from Mongodb

In [0]:
%pip install vaderSentiment
%pip install pysparkling
%pip install h2o
%pip install h2o-pysparkling-3.2

Python interpreter will be restarted.
Requirement already satisfied: vaderSentiment in /databricks/python3/lib/python3.8/site-packages (3.3.2)
Requirement already satisfied: requests in /databricks/python3/lib/python3.8/site-packages (from vaderSentiment) (2.25.1)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests->vaderSentiment) (2020.12.5)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests->vaderSentiment) (4.0.0)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests->vaderSentiment) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests->vaderSentiment) (2.10)
Python interpreter will be restarted.
Python interpreter will be restarted.
Requirement already satisfied: pysparkling in /databricks/python3/lib/python3.8/site-packages (0.6.1)
Requirement already satisfied: future>=0.15 in /databricks/python3/lib/python3.8/site-packages (from pysparkling) (0.18.2)
Requirement already satisfied: boto>=2.36.0 in /databricks/python3/lib/python3.8/site-packages (from pysparkling) (2.49.0)
Requirement already satisfied: requests>=2.6.0 in /databricks/python3/lib/python3.8/site-packages (from pysparkling) (2.25.1)
Requirement already satisfied: pytz>=2019.3 in /databricks/python3/lib/python3.8/site-packages (from pysparkling) (2020.5)
Requirement already satisfied: python-dateutil>=2.8.0 in /databricks/python3/lib/python3.8/site-packages (from pysparkling) (2.8.1)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.8/site-packages (from python-dateutil>=2.8.0->pysparkling) (1.15.0)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.6.0->pysparkling) (2020.12.5)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.6.0->pysparkling) (4.0.0)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.6.0->pysparkling) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.6.0->pysparkling) (2.10)
Python interpreter will be restarted.
Python interpreter will be restarted.
Requirement already satisfied: h2o in /databricks/python3/lib/python3.8/site-packages (3.36.0.3)
Requirement already satisfied: future in /databricks/python3/lib/python3.8/site-packages (from h2o) (0.18.2)
Requirement already satisfied: tabulate in /databricks/python3/lib/python3.8/site-packages (from h2o) (0.8.9)
Requirement already satisfied: requests in /databricks/python3/lib/python3.8/site-packages (from h2o) (2.25.1)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests->h2o) (2020.12.5)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests->h2o) (4.0.0)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests->h2o) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests->h2o) (2.10)
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting h2o-pysparkling-3.2
 Using cached h2o_pysparkling_3.2-3.36.0.3.post1-py2.py3-none-any.whl
Requirement already satisfied: future in /databricks/python3/lib/python3.8/site-packages (from h2o-pysparkling-3.2) (0.18.2)
Requirement already satisfied: tabulate in /databricks/python3/lib/python3.8/site-packages (from h2o-pysparkling-3.2) (0.8.9)
Requirement already satisfied: requests in /databricks/python3/lib/python3.8/site-packages (from h2o-pysparkling-3.2) (2.25.1)
Requirement already satisfied: certifi>=2017.4.17 in /databric

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.ml.feature import Tokenizer, StopWordsRemover
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pysparkling import *
import h2o

In [0]:

spark = SparkSession.builder.config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
                            .config("spark.jars.packages", "ai.h2o:sparkling-water-package_2.12:3.36.0.3-1-3.2")\
                            .config("spark.network.timeout", "36000s")\
                            .config("spark.executor.heartbeatInterval", "3600s")\
                            .config("spark.ext.h2o.fail.on.unsupported.spark.param", "false")\
                            .config("spark.dynamicAllocation.enabled", "false")\
                            .getOrCreate()

In [0]:
database = 'MSDS697'
collection = 'Tweets'
user_name = 'qliu46'
password = 'Tina0726'
address = 'msds697.us6ly.mongodb.net'#grad address from MongoDB
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
df = spark.read.format("mongo").option("uri",connection_string).load().dropna(how='any')

## Create Unique ID

In [0]:
df = df.select("*").withColumn("id", monotonically_increasing_id()).drop('_id')
df.show(3)

+--------------------+---------+-------------------+-------------------+----------+-----------+-----------+-------------+--------------------+---+
 Adj Close| Volumn| created_date| date|like_count|quote_count|reply_count|retweet_count| text| id|
+--------------------+---------+-------------------+-------------------+----------+-----------+-----------+-------------+--------------------+---+
0.002375999931246...| 84321565|2020-01-21 00:00:00|2020-01-21 00:00:00| 0| 0| 0| 4|RT @unlockbc: Blo...| 0|
0.002490000100806...|117752841|2020-01-17 00:00:00|2020-01-17 00:00:00| 0| 0| 0| 3|RT @stephenajulu:...| 1|
0.002375999931246...| 84321565|2020-01-21 00:00:00|2020-01-21 00:00:00| 0| 0| 0| 4|RT @unlockbc: Blo...| 2|
+--------------------+---------+-------------------+-------------------+----------+-----------+-----------+-------------+--------------------+---+
only showing top 3 rows

## Clean tweet text

In [0]:
tweet = df.select('id','text')
tweet = tweet.distinct()

### 1. Remove return handles, handles, url, special characters, numbers and punctuations

In [0]:
def cleanTweet(txt):
    pattern = r'RT @[\w]*:'
    txt = re.sub(pattern, '', txt)
    
    pattern = r'@[\w]*'
    txt = re.sub(pattern, '', txt)
    
    pattern = r'https?://[A-Za-z0-9./]*'
    txt = re.sub(pattern, '', txt)
    
    pattern = r'[^A-Za-z]+'
    txt = re.sub(pattern, ' ', txt)
    return txt.strip()
udf_tweet_clean = udf(cleanTweet, StringType())

In [0]:
# Clean text
tweet_clean = tweet.withColumn('clean_text', udf_tweet_clean('text'))
tweet_clean.show(3)

+---+--------------------+--------------------+
 id| text| clean_text|
+---+--------------------+--------------------+
 1|RT @stephenajulu:...|A must read Why B...|
 0|RT @unlockbc: Blo...|Blockchain video ...|
 3|A must read, Why ...|A must read Why B...|
+---+--------------------+--------------------+
only showing top 3 rows

### 2. Tokenize

In [0]:
# Tokenize text
tokenizer = Tokenizer(inputCol='clean_text', outputCol='words_token')
words_token = tokenizer.transform(tweet_clean)
words_token.show(3)

+---+--------------------+--------------------+--------------------+
 id| text| clean_text| words_token|
+---+--------------------+--------------------+--------------------+
 1|RT @stephenajulu:...|A must read Why B...|[a, must, read, w...|
 0|RT @unlockbc: Blo...|Blockchain video ...|[blockchain, vide...|
 3|A must read, Why ...|A must read Why B...|[a, must, read, w...|
+---+--------------------+--------------------+--------------------+
only showing top 3 rows

### 3. Remove stop words

In [0]:
# Remove stop words
remover = StopWordsRemover(inputCol='words_token', outputCol='words_no_stop')
words_no_stopw = remover.transform(words_token)
words_no_stopw.show(3)

+---+--------------------+--------------------+--------------------+--------------------+
 id| text| clean_text| words_token| words_no_stop|
+---+--------------------+--------------------+--------------------+--------------------+
669|Are BitMEX #bitco...|Are BitMEX bitcoi...|[are, bitmex, bit...|[bitmex, bitcoin,...|
737|$BTC Going up sig...|BTC Going up sign...|[btc, going, up, ...|[btc, going, sign...|
777|RT @lekkercryptis...|Meer dan Duitse b...|[meer, dan, duits...|[meer, dan, duits...|
+---+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows

### 4. Convert words list back to string

In [0]:
def words_to_text(words):
    return ' '.join(words)
udf_words_to_text = udf(words_to_text, StringType())

In [0]:
new_tweet = words_no_stopw.withColumn('new_text', udf_words_to_text('words_no_stop'))
new_tweet.show(3)

+---+--------------------+--------------------+--------------------+--------------------+--------------------+
 id| text| clean_text| words_token| words_no_stop| new_text|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+
669|Are BitMEX #bitco...|Are BitMEX bitcoi...|[are, bitmex, bit...|[bitmex, bitcoin,...|bitmex bitcoin wh...|
737|$BTC Going up sig...|BTC Going up sign...|[btc, going, up, ...|[btc, going, sign...|btc going signal ...|
777|RT @lekkercryptis...|Meer dan Duitse b...|[meer, dan, duits...|[meer, dan, duits...|meer dan duitse b...|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows

## Compute Sentiment Scores of the string

In [0]:
# sentiment score
def ss(text):
    sia = SentimentIntensityAnalyzer()
    ps = sia.polarity_scores(text)
    score = ps['compound']
    return score
udf_ss = udf(ss, FloatType())

In [0]:
sentiment = new_tweet.withColumn('sentiment_score', udf_ss('new_text'))
sentiment = sentiment.select('id','sentiment_score')
sentiment.show(5)

+---+---------------+
 id|sentiment_score|
+---+---------------+
 1| 0.25|
 4| 0.0|
 0| 0.4215|
 3| 0.25|
 2| 0.4215|
+---+---------------+
only showing top 5 rows

## Join all the features

In [0]:
df_new = df.join(sentiment, sentiment.id == df.id).select(['Adj Close','like_count','quote_count','reply_count','retweet_count','sentiment_score'])
df_new.show(5)

+--------------------+----------+-----------+-----------+-------------+---------------+
 Adj Close|like_count|quote_count|reply_count|retweet_count|sentiment_score|
+--------------------+----------+-----------+-----------+-------------+---------------+
0.002375999931246...| 0| 0| 0| 0| -0.3818|
0.002920000115409...| 0| 0| 0| 0| 0.0|
0.003231999929994...| 0| 0| 0| 24| 0.2732|
0.002356000011786...| 1| 0| 0| 0| 0.2263|
 0.00227600010111928| 0| 0| 0| 0| 0.0|
+--------------------+----------+-----------+-----------+-------------+---------------+
only showing top 5 rows

In [0]:
df_new = df_new.withColumn('Adj Close(1000)', df_new['Adj Close']*1000)

In [0]:
df_new.show(5)

+--------------------+----------+-----------+-----------+-------------+---------------+------------------+
 Adj Close|like_count|quote_count|reply_count|retweet_count|sentiment_score| Adj Close(1000)|
+--------------------+----------+-----------+-----------+-------------+---------------+------------------+
0.002375999931246...| 0| 0| 0| 0| -0.3818|2.3759999312460423|
0.002920000115409...| 0| 0| 0| 0| 0.0|2.9200001154094934|
0.003231999929994...| 0| 0| 0| 24| 0.2732|3.2319999299943447|
0.002356000011786...| 1| 0| 0| 0| 0.2263|2.3560000117868185|
 0.00227600010111928| 0| 0| 0| 0| 0.0| 2.27600010111928|
+--------------------+----------+-----------+-----------+-------------+---------------+------------------+
only showing top 5 rows

In [0]:
df_new.printSchema()

root
-- Adj Close: double (nullable = true)
-- like_count: integer (nullable = true)
-- quote_count: integer (nullable = true)
-- reply_count: integer (nullable = true)
-- retweet_count: integer (nullable = true)
-- sentiment_score: float (nullable = true)
-- Adj Close(1000): double (nullable = true)

# Random Forest Regressor - Summer

### Step 1: Sentiment score computation - Continued from the data cleanning process above

In [0]:
df = df_new.drop('Adj Close')

In [0]:
df.show(5)

+----------+-----------+-----------+-------------+---------------+------------------+
like_count|quote_count|reply_count|retweet_count|sentiment_score| Adj Close(1000)|
+----------+-----------+-----------+-------------+---------------+------------------+
 0| 0| 0| 0| -0.3818|2.3759999312460423|
 0| 0| 0| 0| 0.0|2.9200001154094934|
 0| 0| 0| 24| 0.2732|3.2319999299943447|
 1| 0| 0| 0| 0.2263|2.3560000117868185|
 0| 0| 0| 0| 0.0| 2.27600010111928|
+----------+-----------+-----------+-------------+---------------+------------------+
only showing top 5 rows

In [0]:
# smooth and then take log to compress the data, making them dense
from pyspark.sql.functions import log
log_df = df
for col in df.columns[:-1]:
    log_df = log_df.withColumn("log_"+col, log(df[col]+1)).drop(col)
log_df.show(5)

+------------------+------------------+---------------+---------------+------------------+-------------------+
 Adj Close(1000)| log_like_count|log_quote_count|log_reply_count| log_retweet_count|log_sentiment_score|
+------------------+------------------+---------------+---------------+------------------+-------------------+
2.3759999312460423| 0.0| 0.0| 0.0| 0.0|-0.4809432426506699|
2.9200001154094934| 0.0| 0.0| 0.0| 0.0| 0.0|
3.2319999299943447| 0.0| 0.0| 0.0|3.2188758248682006|0.24153344399037374|
2.3560000117868185|0.6931471805599453| 0.0| 0.0| 0.0|0.20400150671906453|
 2.27600010111928| 0.0| 0.0| 0.0| 0.0| 0.0|
+------------------+------------------+---------------+---------------+------------------+-------------------+
only showing top 5 rows

In [0]:
log_df = log_df.distinct()
log_df.count()

Out[22]: 5890

### Step 2: Create Feature Vectors

In [0]:
from pyspark.ml.feature import VectorAssembler

input_cols=['log_like_count','log_quote_count','log_reply_count','log_retweet_count','log_sentiment_score']
va = VectorAssembler(outputCol="features", inputCols=input_cols)
lpoints = va.transform(log_df)

In [0]:
lpoints.show(3)

+------------------+--------------+---------------+---------------+-----------------+--------------------+--------------------+
 Adj Close(1000)|log_like_count|log_quote_count|log_reply_count|log_retweet_count| log_sentiment_score| features|
+------------------+--------------+---------------+---------------+-----------------+--------------------+--------------------+
2.4429999757558107| 0.0| 0.0| 0.0| 0.0| 0.5066368130388528|(5,[4],[0.5066368...|
 2.432999899610877| 0.0| 0.0| 0.0| 0.0| 0.6694178364776734|(5,[4],[0.6694178...|
1.9970000721514225| 0.0| 0.0| 0.0| 0.0|-0.31910399074742923|(5,[4],[-0.319103...|
+------------------+--------------+---------------+---------------+-----------------+--------------------+--------------------+
only showing top 3 rows

In [0]:
lpoints = lpoints.select("features", "Adj Close(1000)").withColumnRenamed("Adj Close(1000)", "label")
lpoints.show(3, False)

+------------------------------+------------------+
features |label |
+------------------------------+------------------+
(5,[4],[0.5066368130388528]) |2.4429999757558107|
(5,[4],[0.6694178364776734]) |2.432999899610877 |
(5,[4],[-0.31910399074742923])|1.9970000721514225|
+------------------------------+------------------+
only showing top 3 rows

### Step 3: Train Test Split

In [0]:
#Divide the dataset into training and vaildation sets.
splits = lpoints.randomSplit([0.8, 0.2],1)

sentiment_train = splits[0]
sentiment_valid = splits[1]

### Step 4: Normalize data

In [0]:
from pyspark.ml.feature import StandardScaler
standardscaler = StandardScaler(withStd=True, withMean=True, inputCol='features', outputCol='scaled_features')
train_scaled = standardscaler.fit(sentiment_train).transform(sentiment_train).select('scaled_features', 'label').withColumnRenamed('scaled_features', 'features')

test_scaled = standardscaler.fit(sentiment_train).transform(sentiment_valid)\
              .select('scaled_features', 'label')\
              .withColumnRenamed('scaled_features', 'features')

train_scaled.show(3, truncate=False)

+------------------------------------------------------------------------------------------------------+------------------+
features |label |
+------------------------------------------------------------------------------------------------------+------------------+
[-0.513301125925375,-0.14328159095037388,-0.25130662092850425,-0.5544432193991585,-0.1028395562368843]|1.5369999455288053|
[-0.513301125925375,-0.14328159095037388,-0.25130662092850425,-0.5544432193991585,-0.1028395562368843]|1.5660000499337912|
[-0.513301125925375,-0.14328159095037388,-0.25130662092850425,-0.5544432193991585,-0.1028395562368843]|1.5869999770075083|
+------------------------------------------------------------------------------------------------------+------------------+
only showing top 3 rows

In [0]:
# train_scaled.write.saveAsTable("train_scaled")
# test_scaled.write.saveAsTable("test_scaled")

### Step 5: Train the Model

In [0]:
%%time
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(maxDepth=20)
rfmodel = rf.fit(train_scaled)
# print(rfmodel.toDebugString)

CPU times: user 579 ms, sys: 229 ms, total: 808 ms
Wall time: 3min 50s

### Step 6: Interpret the model

In [0]:
%%time
validpredicts = rfmodel.transform(test_scaled)
validpredicts.show()

+--------------------+------------------+------------------+
 features| label| prediction|
+--------------------+------------------+------------------+
[-0.5133011259253...|1.6850000247359276|2.2928419003540217|
[-0.5133011259253...|1.8110000528395176|2.2928419003540217|
[-0.5133011259253...|1.8309999722987413|2.2928419003540217|
[-0.5133011259253...|1.8339999951422215|2.2928419003540217|
[-0.5133011259253...| 1.851000008173287|2.2928419003540217|
[-0.5133011259253...| 1.981999957934022|2.2928419003540217|
[-0.5133011259253...|2.0089999306946993|2.2928419003540217|
[-0.5133011259253...| 2.010999945923686|2.2928419003540217|
[-0.5133011259253...| 2.102999947965145|2.2928419003540217|
[-0.5133011259253...| 2.145000034943223|2.2928419003540217|
[-0.5133011259253...|2.2559999488294125|2.2928419003540217|
[-0.5133011259253...|2.2750000935047865|2.2928419003540217|
[-0.5133011259253...| 2.305000089108944|2.2928419003540217|
[-0.5133011259253...| 2.355000004172325|2.2928419003540217|
[-0.5133011259253...| 2.394000068306923|2.2928419003540217|
[-0.5133011259253...|2.3980000987648964|2.2928419003540217|
[-0.5133011259253...|2.4500000290572643|2.2928419003540217|
[-0.5133011259253...|2.4629998952150345|2.2928419003540217|
[-0.5133011259253...| 2.481000032275915|2.2928419003540217|
[-0.5133011259253...| 2.495999913662672|2.2928419003540217|
+--------------------+------------------+------------------+
only showing top 20 rows

CPU times: user 749 ms, sys: 265 ms, total: 1.01 s
Wall time: 4min 1s

### Step 7: Evaluate the model

In [0]:
#Evaluate the model. default metric : Area Under ROC
from pyspark.ml.evaluation import RegressionEvaluator
reg_evaluator = RegressionEvaluator()
print (reg_evaluator.getMetricName() +":" + str(reg_evaluator.evaluate(validpredicts)))

rmse:0.36405564150278014

In [0]:
reg_evaluator.setMetricName('r2')
print (reg_evaluator.getMetricName() +":" + str(reg_evaluator.evaluate(validpredicts)))

r2:-0.027149781106798665

In [0]:
reg_evaluator.setMetricName('mse')
print (reg_evaluator.getMetricName() +":" + str(reg_evaluator.evaluate(validpredicts)))

mse:0.13253651011000078

In [0]:
ss.stop()